### Before start, please replace every folder path

In [1]:
import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from ase import Atoms
from ase.io import read
from ase.io import write
from ase.visualize import view
import itertools
import time

%matplotlib inline

pd.set_option('display.max_columns', 500)

#### panalty vs time for different penalty factor

In [15]:
from utilities import *
from classes import *

lattice=pickle.load(open('no1NN_T4frac50/lattice.p','rb'))
structure=pickle.load(open('no1NN_T4frac50/structure.p','rb'))
penalty=pickle.load(open('no1NN_T4frac50/penalty.p', 'rb'))

In [11]:
swap_str_vec_py_template = '''import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from ase import Atoms
from ase.io import read
from ase.io import write
from ase.visualize import view
import itertools
import time

from utilities import *
from classes import *

lattice=pickle.load(open('lattice.p','rb'))
structure=pickle.load(open('structure.p','rb'))
penalty=pickle.load(open('penalty.p', 'rb'))
Al_num = 243
num_vecs = 1
num_steps = 1000000
penalty_factor = {0}

str_vecs = structure.random_config_swap(atom_num=Al_num, penalty=penalty, num_vecs=num_vecs, num_step=num_steps, burn_in_period=0, ptfile='f20')


'''

In [12]:
swap_qscript_template = '''#!/bin/bash
#$ -q *@@crc
#$ -pe smp 1
python J_swap_pt.py
'''

In [13]:
filename = 'no1NN/J_swap_pt.py'
with open(filename, 'w+') as file:
    file.write(swap_str_vec_py_template)
filename = 'no1NN/J_swap_pt.sh'
with open(filename, 'w+') as file:
    file.write(swap_qscript_template)

#### swap with 5000 iterations no 1NN

In [216]:
no1NN_py_template = '''import pandas as pd
import numpy as np
import copy
from math import *
import os
import random
from collections import defaultdict
import csv
from functools import reduce
from copy import deepcopy
import pickle
from timeit import default_timer as timer
from matplotlib import pyplot as plt
from ase import Atoms
from ase.io import read
from ase.io import write
from ase.visualize import view
import itertools
import time

from utilities import *
from classes import *

lattice=pickle.load(open('lattice.p','rb'))
structure=pickle.load(open('structure.p','rb'))
penalty=pickle.load(open('penalty.p', 'rb'))

Al_num = int(round(1/(1+{1})*972))
num_vecs = 1
num_steps = 5010
penalty_factor = 1
prob = dict()
prob['4']={0}/100

str_vecs = []
for i in range(20):
    str_vec = structure.random_config_swap(atom_num=Al_num, prob=prob, penalty=penalty, num_vecs=num_vecs, num_step=num_steps, burn_in_period=5000)[0]
    str_vecs.append(str_vec)
    
pickle.dump(str_vecs, open('swap_str_vecs_no1NNnoT4T4_T4frac{0}_{1}_{2}.p','wb'))
    
titrate_result = structure.titrate_clusters_multi_configs(str_vecs, titration_groups=[['2-27', '2-20']],titrate_num=100)
final_result = dict()
final_result['2-27'] = titrate_result['2-27']
final_result['2-20'] = titrate_result['2-20']
pickle.dump(final_result, open('swap_no1NNnoT4T4_T4frac{0}_titration_T4T4_T1T4_{1}_{2}.p','wb'))


exc3Al_result3 = structure.titrate_clusters_multi_configs(str_vecs, titration_groups=[['2-27', '2-20']],excluding_types=['3-44'], titrate_num=1)
final_result3 = dict()
final_result3['2-27'] = exc3Al_result3['2-27']
final_result3['2-20'] = exc3Al_result3['2-20']
pickle.dump(final_result3, open('swap_no1NNnoT4T4_T4frac{0}_titration_T4T4_T1T4_EX3Al_{1}_{2}.p','wb'))


counting_types = ['1-1','1-2','1-3','1-4','2-20','2-27']
count_result = structure.count_clusters_multi_configs(str_vecs, counting_types=counting_types)
count_final_result = dict()
for t in counting_types:
    count_final_result[t] = count_result[t]
pickle.dump(count_final_result, open('swap_no1NNnoT4T4_T4frac{0}_count_T4T4_T1T4_{1}_{2}.p','wb'))

count_result2 = structure.count_clusters_multi_configs(str_vecs, counting_types=counting_types,excluding_types=['3-44'])
count_final_result2 = dict()
for t in counting_types:
    count_final_result2[t] = count_result2[t]
pickle.dump(count_final_result2, open('swap_no1NNnoT4T4_T4frac{0}_count_T4T4_T1T4_EX3Al_{1}_{2}.p','wb'))

'''

In [217]:
no1NN_qscript_template = '''#!/bin/bash
#$ -q *@@crc
#$ -pe smp 1
python J_{0}_{1}_{2}.py
'''

In [218]:
ratios = [15]
fracs = [75,80,85,90,95]

In [219]:
for frac in fracs:
    for ratio in ratios:
        for i in range(1,51):
            filename = 'no1NNnoT4T4_T4varfrac/J_{}_{}_{}.py'.format(frac, ratio,i)
            with open(filename, 'w+') as file:
                file.write(no1NN_py_template.format(frac, ratio,i))
            filename =  'no1NNnoT4T4_T4varfrac/J_{}_{}_{}.sh'.format(frac,ratio,i)
            with open(filename, 'w+') as file:
                file.write(no1NN_qscript_template.format(frac,ratio,i))    

In [220]:
ratios = [9]
fracs = [40,45,50,55,60]

In [221]:
for frac in fracs:
    for ratio in ratios:
        for i in range(1,51):
            filename = 'no1NNnoT4T4_T4varfrac/J_{}_{}_{}.py'.format(frac, ratio,i)
            with open(filename, 'w+') as file:
                file.write(no1NN_py_template.format(frac, ratio,i))
            filename =  'no1NNnoT4T4_T4varfrac/J_{}_{}_{}.sh'.format(frac,ratio,i)
            with open(filename, 'w+') as file:
                file.write(no1NN_qscript_template.format(frac,ratio,i))    

### Si/Al=9 no1NN T4frac50 results

In [115]:
ratios = [9]

In [116]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NN count T4T4 & T1T4

In [117]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [118]:
result_9 = pickle.load(open('no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_9_summary.p','rb'))

In [123]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.1517938144329897

In [120]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

In [121]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.043979381443298972

In [122]:
np.mean(result_9['1-4'])

48.0

### no1NN count T4T4 & T1T4 excluding 3Al

In [124]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [125]:
result_9 = pickle.load(open('no1NN_T4frac50/swap_no1NN_T4frac50_count_T4T4_T1T4_EX3Al_9_summary.p','rb'))

In [126]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.14868041237113402

In [127]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

In [128]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.040865979381443297

In [129]:
np.mean(result_9['1-4'])/int(round(1/(1+9)*972))

0.49328865979381442

In [169]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [170]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NN titrate T4T4

In [130]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [132]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

### no1NN titrate T4T4 Exc3Al

In [133]:
types = ratios
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [134]:
temp = summarize_titrate_results(results, both=False)

In [135]:
np.mean(temp['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

### no1NN titrate T4T4 & T4T1

In [136]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [137]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.15023711340206186

In [138]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

In [139]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.042422680412371135

### no1NN titrate T4T4 & T4T1 Exc3Al

In [140]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac50/swap_no1NN_T4frac50_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [142]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.14868041237113402

In [143]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.10781443298969072

In [144]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.040865979381443297

### Si/Al=9 no1NNnoT4T4 T4frac50 results

In [148]:
ratios = [9]

In [149]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NNnoT4T4 count T4T4 & T1T4

In [150]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [151]:
result_9 = pickle.load(open('no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_9_summary.p','rb'))

In [152]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.076639175257731965

In [153]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.0

In [154]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.076639175257731965

In [155]:
np.mean(result_9['1-4'])

48.0

### no1NNnoT4T4 count T4T4 & T1T4 excluding 3Al

In [156]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [157]:
result_9 = pickle.load(open('no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_count_T4T4_T1T4_EX3Al_9_summary.p','rb'))

In [158]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.071113402061855666

In [159]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.0

In [160]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.071113402061855666

In [161]:
np.mean(result_9['1-4'])/int(round(1/(1+9)*972))

0.49208247422680412

In [169]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [170]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NNnoT4T4 titrate T4T4 & T4T1

In [164]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [165]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.073876288659793815

In [166]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.0

In [167]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.073876288659793815

### no1NNnoT4T4 titrate T4T4 & T4T1 Exc3Al

In [168]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac50/swap_no1NNnoT4T4_T4frac50_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [169]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.071113402061855666

In [170]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.0

In [171]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.071113402061855666

### Si/Al=9 No1NN results

In [177]:
ratios = [9]

In [146]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NN count T4T4 & T1T4

In [178]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN/swap_no1NN_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN/swap_no1NN_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [179]:
result_9 = pickle.load(open('no1NN/swap_no1NN_count_T4T4_T1T4_9_summary.p','rb'))

In [180]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.029680412371134019

In [181]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.0053711340206185568

In [182]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.024309278350515464

In [184]:
np.mean(result_9['1-4'])/int(round(1/(1+9)*972))

0.11099999999999999

### no1NN count T4T4 & T1T4 excluding 3Al

In [185]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN/swap_no1NN_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN/swap_no1NN_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [186]:
result_9 = pickle.load(open('no1NN/swap_no1NN_count_T4T4_T1T4_EX3Al_9_summary.p','rb'))

In [187]:
np.mean(result_9['total 2-body'])/int(round(1/(1+9)*972))

0.026505154639175261

In [188]:
np.mean(result_9['2-27'])/int(round(1/(1+9)*972))

0.0053711340206185568

In [189]:
np.mean(result_9['2-20'])/int(round(1/(1+9)*972))

0.021134020618556699

In [190]:
np.mean(result_9['1-4'])/int(round(1/(1+9)*972))

0.10941237113402061

In [169]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [170]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NN titrate T4T4 & T4T1

In [193]:
types = ratios[:1]
num = 50
prefix = 'no1NN/swap_no1NN_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN/swap_no1NN_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [194]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.02809278350515464

In [195]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.0053711340206185568

In [196]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.022721649484536085

### no1NN titrate T4T4 & T4T1 Exc3Al

In [197]:
types = ratios[:1]
num = 50
prefix = 'no1NN/swap_no1NN_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN/swap_no1NN_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [198]:
np.mean(results['total'])/int(round(1/(1+9)*972))

0.026505154639175261

In [199]:
np.mean(results['2-27'])/int(round(1/(1+9)*972))

0.0053711340206185568

In [200]:
np.mean(results['2-20'])/int(round(1/(1+9)*972))

0.021134020618556699

### Si/Al=15results

In [35]:
ratios = [15]

In [36]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NN count T4T4 & T1T4

In [49]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [50]:
result_15 = pickle.load(open('no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_15_summary.p','rb'))

In [51]:
np.mean(result_15['total 2-body'])/(1/(1+15)*972)

0.21086419753086422

In [52]:
np.mean(result_15['2-27'])/int(round(1/(1+15)*972))

0.19624590163934427

In [53]:
np.mean(result_15['2-20'])/int(round(1/(1+15)*972))

0.013754098360655737

In [54]:
np.mean(result_15['1-4'])/int(round(1/(1+15)*972))

0.83606557377049184

### no1NN count T4T4 & T1T4 excluding 3Al

In [55]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [58]:
result = pickle.load(open('no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_15_summary.p','rb'))

In [59]:
np.mean(result['total 2-body'])/int(round(1/(1+15)*972))

0.20960655737704917

In [60]:
np.mean(result['2-27'])/int(round(1/(1+15)*972))

0.19624590163934427

In [61]:
np.mean(result['2-20'])/int(round(1/(1+15)*972))

0.013360655737704917

In [62]:
np.mean(result['1-4'])/int(round(1/(1+15)*972))

0.83586885245901643

In [63]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [64]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NN titrate T4T4

In [65]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [66]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

### no1NN titrate T4T4 Exc3Al

In [102]:
types = ratios
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [103]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

### no1NN titrate T4T4 & T4T1

In [104]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [105]:
np.mean(results['total'])/(1/(1+15)*972)

0.21066666666666667

In [106]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

In [107]:
np.mean(results['2-20'])/(1/(1+15)*972)

0.013613168724279834

### no1NN titrate T4T4 & T4T1 Exc3Al

In [108]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [109]:
np.mean(results['total'])/(1/(1+15)*972)

0.21046913580246912

In [110]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

In [111]:
np.mean(results['2-20'])/(1/(1+15)*972)

0.013415637860082303

### Si/Al=15 No1NNnoT4T4 T4frac84 results

In [75]:
ratios = [15]

In [76]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NNnoT4T4 count T4T4 & T1T4

In [112]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [113]:
result_15 = pickle.load(open('no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_15_summary.p','rb'))

In [114]:
np.mean(result_15['total 2-body'])/int(round(1/(1+15)*972))

0.02577049180327869

In [80]:
np.mean(result_15['2-27'])/int(round(1/(1+15)*972))

0.0

In [81]:
np.mean(result_15['2-20'])/int(round(1/(1+15)*972))

0.02577049180327869

In [82]:
np.mean(result_15['1-4'])/int(round(1/(1+15)*972))

0.83606557377049184

### no1NNnoT4T4 count T4T4 & T1T4 excluding 3Al

In [83]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [84]:
result = pickle.load(open('no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_EX3Al_15_summary.p','rb'))

In [85]:
np.mean(result['total 2-body'])/int(round(1/(1+15)*972))

0.025442622950819675

In [86]:
np.mean(result['2-27'])/int(round(1/(1+15)*972))

0.0

In [87]:
np.mean(result['2-20'])/int(round(1/(1+15)*972))

0.025442622950819675

In [88]:
np.mean(result['1-4'])/int(round(1/(1+15)*972))

0.83590163934426231

In [89]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [90]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NNnoT4T4 titrate T4T4 & T4T1

In [98]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_t
itration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [99]:
np.mean(results['total'])/(1/(1+15)*972)

0.025711934156378603

### no1NNnoT4T4 titrate T4T4 & T4T1 Exc3Al

In [100]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [101]:
np.mean(results['total'])/(1/(1+15)*972)

0.025547325102880658

### Si/Al=15results

In [35]:
ratios = [15]

In [36]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

### no1NN count T4T4 & T1T4

In [49]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [50]:
result_15 = pickle.load(open('no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_15_summary.p','rb'))

In [51]:
np.mean(result_15['total 2-body'])/(1/(1+15)*972)

0.21086419753086422

In [52]:
np.mean(result_15['2-27'])/int(round(1/(1+15)*972))

0.19624590163934427

In [53]:
np.mean(result_15['2-20'])/int(round(1/(1+15)*972))

0.013754098360655737

In [54]:
np.mean(result_15['1-4'])/int(round(1/(1+15)*972))

0.83606557377049184

### no1NN count T4T4 & T1T4 excluding 3Al

In [55]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [58]:
result = pickle.load(open('no1NN_T4frac84/swap_no1NN_T4frac84_count_T4T4_T1T4_EX3Al_15_summary.p','rb'))

In [59]:
np.mean(result['total 2-body'])/int(round(1/(1+15)*972))

0.20960655737704917

In [60]:
np.mean(result['2-27'])/int(round(1/(1+15)*972))

0.19624590163934427

In [61]:
np.mean(result['2-20'])/int(round(1/(1+15)*972))

0.013360655737704917

In [62]:
np.mean(result['1-4'])/int(round(1/(1+15)*972))

0.83586885245901643

In [63]:
def combine_titration_files(prefix, types, num, savefileprefix,both=True):
    for t in types:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(t, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(t),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [64]:
def summarize_titrate_results(results, both=True):
    temp = {}
    if both:
        temp['2-20'] = []
        temp['total'] = []
    temp['2-27'] = []
    for index, item in enumerate(results['2-27']):
        temp['2-27'].append(np.mean(item))
        if both:
            temp['2-20'].append(np.mean(results['2-20'][index]))
            temp['total'].append(np.mean(item)+np.mean(results['2-20'][index]))
    return temp

### no1NN titrate T4T4

In [65]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [66]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

### no1NN titrate T4T4 Exc3Al

In [102]:
types = ratios
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix,both=False)

In [103]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

### no1NN titrate T4T4 & T4T1

In [104]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [105]:
np.mean(results['total'])/(1/(1+15)*972)

0.21066666666666667

In [106]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

In [107]:
np.mean(results['2-20'])/(1/(1+15)*972)

0.013613168724279834

### no1NN titrate T4T4 & T4T1 Exc3Al

In [108]:
types = ratios[:1]
num = 50
prefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NN_T4frac84/swap_no1NN_T4frac84_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [109]:
np.mean(results['total'])/(1/(1+15)*972)

0.21046913580246912

In [110]:
np.mean(results['2-27'])/(1/(1+15)*972)

0.19705349794238683

In [111]:
np.mean(results['2-20'])/(1/(1+15)*972)

0.013415637860082303

### T4 frac var

In [10]:
def combine_count_files(prefix, ratios, types, num, savefileprefix):
    for ratio in ratios:
        results = {}
        for t in types:
            results[t] = []
        results['total 2-body'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            for t in types:
                results[t].extend(temp[t])
            results['total 2-body'].extend([temp['2-20'][j]+temp['2-27'][j] for j,num in enumerate(temp['2-20'])])
        #print(results)
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  

In [11]:
def combine_titration_files(prefix, ratios, num, savefileprefix,both=True):
    for ratio in ratios:
        results = {}
        if both:
            results['2-20'] = []
            results['total'] = []
        results['2-27'] = []
        for i in range(1, num+1):
            filename = prefix.format(ratio, i)
            temp = pickle.load(open(filename, 'rb'))
            if both:
                results['2-20'].extend(temp['2-20'])
                results['total'].extend([temp['2-20'][j]+temp['2-27'][j]for j,num in enumerate(temp['2-20'])])
            results['2-27'].extend(temp['2-27'])
        pickle.dump(results,open(savefileprefix.format(ratio),'wb'))
    return results  
#[k] for k in range(len(temp['2-27'][0])) 

In [14]:
def combine_all_files_frac_var(ratios,types, num, fracs):
    types = types
    num = num
    for frac in fracs:
        prefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_count_T4T4_T1T4_{0}_{1}.p'
        savefileprefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_count_T4T4_T1T4_{0}_summary.p'
        results = combine_count_files(prefix, ratios,types, num, savefileprefix)
        prefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_count_T4T4_T1T4_EX3Al_{0}_{1}.p'
        savefileprefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_count_T4T4_T1T4_EX3Al_{0}_summary.p'
        results = combine_count_files(prefix, ratios, types, num, savefileprefix)
        prefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_titration_T4T4_T1T4_{0}_{1}.p'
        savefileprefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_titration_T4T4_T1T4_{0}_summary.p'
        results = combine_titration_files(prefix, ratios, num, savefileprefix)
        prefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_titration_T4T4_T1T4_EX3Al_{0}_{1}.p'
        savefileprefix = 'no1NNnoT4T4_T4varfrac/swap_no1NNnoT4T4_T4frac'+str(frac)+'_titration_T4T4_T1T4_EX3Al_{0}_summary.p'
        results = combine_titration_files(prefix, ratios, num, savefileprefix)
    return

In [15]:
ratios = [15]
fracs = [75,80,85,90,95]
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num=50
for frac in fracs:
    combine_all_files_frac_var(ratios,types, num, fracs)

ratios = [9]
fracs = [40,45,50,55,60]

for frac in fracs:
    combine_all_files_frac_var(ratios,types, num, fracs)

In [112]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_{}_summary.p'
results = combine_count_files(prefix, ratios,types, num, savefileprefix)

In [83]:
types = ['1-1','1-2','1-3','1-4','2-20','2-27']
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_count_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_count_files(prefix, ratios[:1], types, num, savefileprefix)

In [98]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)

In [100]:
types = ratios[:1]
num = 50
prefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_EX3Al_{}_{}.p'
savefileprefix = 'no1NNnoT4T4_T4frac84/swap_no1NNnoT4T4_T4frac84_titration_T4T4_T1T4_EX3Al_{}_summary.p'
results = combine_titration_files(prefix, types, num, savefileprefix)